In [ ]:
!pip install numpy opencv-python tensorflow

In [ ]:
!pip uninstall tensorflow -y
!pip cache purge
!pip install tensorflow --no-cache-dir

In [ ]:
import os
import sys
import cv2
import numpy as np
import time
import gc
import matplotlib.pyplot as plt

# Ensure TensorFlow is correctly imported
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

# Paths
dataset_path = "E:\Minor_project\FakeVideodetector\DataSet"
model_path = "deepfake_video_model.h5"

# Function to check dataset integrity
def check_dataset(directory):
    if not os.path.exists(directory):
        raise FileNotFoundError(f"Dataset folder not found: {directory}")
    print(f"Dataset {directory} is ready and checked!")

check_dataset(dataset_path)

# Load Model if Exists
def load_or_train_model():
    if os.path.exists(model_path):
        print("Loading pre-trained model...")
        model = load_model(model_path)
    else:
        print("Training new model...")
        model, history = train_model()
        model.save(model_path)
        plot_training_history(history)
    return model

# Training Function
def train_model():
    from tensorflow.keras.layers import Conv3D, Dense, Flatten, Dropout, MaxPooling3D
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.optimizers import Adam

    model = Sequential([
        Conv3D(32, kernel_size=(3,3,3), activation='relu', input_shape=(30, 64, 64, 3)),
        MaxPooling3D(pool_size=(2,2,2)),
        Dropout(0.3),
        Conv3D(64, kernel_size=(3,3,3), activation='relu'),
        MaxPooling3D(pool_size=(2,2,2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    print("Model compiled successfully.")
    
    # Placeholder for actual training code using dataset
    history = None # model.fit(...)
    
    return model, history

# Function to Plot Training History
def plot_training_history(history):
    if history:
        plt.figure(figsize=(12, 5))
        plt.subplot(1, 2, 1)
        plt.plot(history.history['accuracy'], label='Train Accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.title('Model Accuracy')
        
        plt.subplot(1, 2, 2)
        plt.plot(history.history['loss'], label='Train Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.title('Model Loss')
        
        plt.show()

# Process Video for Prediction
def predict_video(video_path, model):
    if not os.path.exists(video_path):
        print("Video file not found!")
        return
    
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    predictions = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame = cv2.resize(frame, (64, 64))
        frame = img_to_array(frame) / 255.0
        predictions.append(frame)
        frame_count += 1
        
        if frame_count >= 30:
            break  # Taking only the first 30 frames
    
    cap.release()
    cv2.destroyAllWindows()
    
    if len(predictions) < 30:
        print("Not enough frames detected!")
        return
    
    predictions = np.array(predictions).reshape(1, 30, 64, 64, 3)
    result = model.predict(predictions)
    confidence = result[0][0] * 100
    print(f"Prediction Confidence: {confidence:.2f}%")
    print("Fake Video" if result[0][0] > 0.5 else "Real Video.")
    
# Main Execution
if __name__ == "__main__":
    model = load_or_train_model()
    video_file = "E:\Minor_project\FakeVideodetector\INDvsPAK_Rivalry.mp4"  # Change to actual video file
    predict_video(r"E:\Minor_project\FakeVideodetector\office-dance-practice-1.mp4", model)
    predict_video(video_file, model)
    predict_video(r"E:\Minor_project\FakeVideodetector\Sumanth_patel.mp4", model)
    
    # Clear Memory
gc.collect()
